In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm; tqdm.pandas()
pd.options.display.max_columns = 202
pd.options.display.max_rows = 300

In [2]:
train = pd.read_pickle('data/train_original.pkl')
test = pd.read_pickle('data/test_original.pkl')
special_cols = [col for col in train.columns if train[col].dtype != np.float64]
feature_cols = [col for col in train.columns if col not in special_cols]

In [3]:
def add_noise(series, noise_level): 
    return series * (1 + noise_level * np.random.randn(len(series)))
                     
def target_encode(trn_series=None, 
                  tst_series=None, 
                  target=None, 
                  min_samples_leaf=1, 
                  smoothing=1,
                  noise_level=0):
                      
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [4]:
cum_target_encode_train = pd.DataFrame()
cum_target_encode_test = pd.DataFrame()

for i, col in tqdm(enumerate(feature_cols)):
    selected_features = feature_cols[:i+1]
    
    flatten_train = pd.DataFrame()
    flatten_train['original_values'] = np.concatenate(train[selected_features].values)
    flatten_train['target'] = np.concatenate([[t]*len(selected_features) for t in train.target])
    
    flatten_test = pd.DataFrame()
    flatten_test['original_values'] = np.concatenate(test[selected_features].values)

    trn_me, tst_me = target_encode(flatten_train['original_values'], 
                                    flatten_test['original_values'], 
                                    target=flatten_train.target,  
                                    min_samples_leaf=100,
                                    smoothing=10,
                                    noise_level=0.01)
    
    flatten_train['target_enc'] = trn_me.values
    flatten_test['target_enc'] = tst_me.values
    trn_me_mapping = flatten_train.groupby('original_values')['target_enc'].mean()
    tst_me_mapping = flatten_test.groupby('original_values')['target_enc'].mean()

    cum_target_encode_train[col+'_cumulative_me'] = train[col].map(trn_me_mapping)
    cum_target_encode_test[col+'_cumulative_me'] = test[col].map(tst_me_mapping)  
    
cum_target_encode_train.head(3)

200it [4:39:26, 166.37s/it]


,var_0_cumulative_me,var_1_cumulative_me,var_2_cumulative_me,var_3_cumulative_me,var_4_cumulative_me,var_5_cumulative_me,var_6_cumulative_me,var_7_cumulative_me,var_8_cumulative_me,var_9_cumulative_me,var_10_cumulative_me,var_11_cumulative_me,var_12_cumulative_me,var_13_cumulative_me,var_14_cumulative_me,var_15_cumulative_me,var_16_cumulative_me,var_17_cumulative_me,var_18_cumulative_me,var_19_cumulative_me,var_20_cumulative_me,var_21_cumulative_me,var_22_cumulative_me,var_23_cumulative_me,var_24_cumulative_me,var_25_cumulative_me,var_26_cumulative_me,var_27_cumulative_me,var_28_cumulative_me,var_29_cumulative_me,var_30_cumulative_me,var_31_cumulative_me,var_32_cumulative_me,var_33_cumulative_me,var_34_cumulative_me,var_35_cumulative_me,var_36_cumulative_me,var_37_cumulative_me,var_38_cumulative_me,var_39_cumulative_me,var_40_cumulative_me,var_41_cumulative_me,var_42_cumulative_me,var_43_cumulative_me,var_44_cumulative_me,var_45_cumulative_me,var_46_cumulative_me,var_47_cumulative_me,var_48_cumulative_me,var_49_cumulative_me,var_50_cumulative_me,var_51_cumulative_me,var_52_cumulative_me,var_53_cumulative_me,var_54_cumulative_me,var_55_cumulative_me,var_56_cumulative_me,var_57_cumulative_me,var_58_cumulative_me,var_59_cumulative_me,var_60_cumulative_me,var_61_cumulative_me,var_62_cumulative_me,var_63_cumulative_me,var_64_cumulative_me,var_65_cumulative_me,var_66_cumulative_me,var_67_cumulative_me,var_68_cumulative_me,var_69_cumulative_me,var_70_cumulative_me,var_71_cumulative_me,var_72_cumulative_me,var_73_cumulative_me,var_74_cumulative_me,var_75_cumulative_me,var_76_cumulative_me,var_77_cumulative_me,var_78_cumulative_me,var_79_cumulative_me,var_80_cumulative_me,var_81_cumulative_me,var_82_cumulative_me,var_83_cumulative_me,var_84_cumulative_me,var_85_cumulative_me,var_86_cumulative_me,var_87_cumulative_me,var_88_cumulative_me,var_89_cumulative_me,var_90_cumulative_me,var_91_cumulative_me,var_92_cumulative_me,var_93_cumulative_me,var_94_cumulative_me,var_95_cumulative_me,var_96_cumulative_me,var_97_cumulative_me,var_98_cumulative_me,var_99_cumulative_me,var_100_cumulative_me,var_101_cumulative_me,var_102_cumulative_me,var_103_cumulative_me,var_104_cumulative_me,var_105_cumulative_me,var_106_cumulative_me,var_107_cumulative_me,var_108_cumulative_me,var_109_cumulative_me,var_110_cumulative_me,var_111_cumulative_me,var_112_cumulative_me,var_113_cumulative_me,var_114_cumulative_me,var_115_cumulative_me,var_116_cumulative_me,var_117_cumulative_me,var_118_cumulative_me,var_119_cumulative_me,var_120_cumulative_me,var_121_cumulative_me,var_122_cumulative_me,var_123_cumulative_me,var_124_cumulative_me,var_125_cumulative_me,var_126_cumulative_me,var_127_cumulative_me,var_128_cumulative_me,var_129_cumulative_me,var_130_cumulative_me,var_131_cumulative_me,var_132_cumulative_me,var_133_cumulative_me,var_134_cumulative_me,var_135_cumulative_me,var_136_cumulative_me,var_137_cumulative_me,var_138_cumulative_me,var_139_cumulative_me,var_140_cumulative_me,var_141_cumulative_me,var_142_cumulative_me,var_143_cumulative_me,var_144_cumulative_me,var_145_cumulative_me,var_146_cumulative_me,var_147_cumulative_me,var_148_cumulative_me,var_149_cumulative_me,var_150_cumulative_me,var_151_cumulative_me,var_152_cumulative_me,var_153_cumulative_me,var_154_cumulative_me,var_155_cumulative_me,var_156_cumulative_me,var_157_cumulative_me,var_158_cumulative_me,var_159_cumulative_me,var_160_cumulative_me,var_161_cumulative_me,var_162_cumulative_me,var_163_cumulative_me,var_164_cumulative_me,var_165_cumulative_me,var_166_cumulative_me,var_167_cumulative_me,var_168_cumulative_me,var_169_cumulative_me,var_170_cumulative_me,var_171_cumulative_me,var_172_cumulative_me,var_173_cumulative_me,var_174_cumulative_me,var_175_cumulative_me,var_176_cumulative_me,var_177_cumulative_me,var_178_cumulative_me,var_179_cumulative_me,var_180_cumulative_me,var_181_cumulative_me,var_182_cumulative_me,var_183_cumulative_me,var_184_cumulative_me,var_185_cumulative_me,var_186_cumulativ

In [5]:
feature_cols = cum_target_encode_train.columns.tolist()

In [6]:
for df in [cum_target_encode_train, cum_target_encode_test]:
    df['target_encode_max'] = df[feature_cols].max(axis=1)
    df['target_encode_min'] = df[feature_cols].min(axis=1)
    df['target_encode_sum'] = df[feature_cols].sum(axis=1)
    df['target_encode_std'] = df[feature_cols].std(axis=1)

In [7]:
cum_target_encode_train.head(3)

,var_0_cumulative_me,var_1_cumulative_me,var_2_cumulative_me,var_3_cumulative_me,var_4_cumulative_me,var_5_cumulative_me,var_6_cumulative_me,var_7_cumulative_me,var_8_cumulative_me,var_9_cumulative_me,var_10_cumulative_me,var_11_cumulative_me,var_12_cumulative_me,var_13_cumulative_me,var_14_cumulative_me,var_15_cumulative_me,var_16_cumulative_me,var_17_cumulative_me,var_18_cumulative_me,var_19_cumulative_me,var_20_cumulative_me,var_21_cumulative_me,var_22_cumulative_me,var_23_cumulative_me,var_24_cumulative_me,var_25_cumulative_me,var_26_cumulative_me,var_27_cumulative_me,var_28_cumulative_me,var_29_cumulative_me,var_30_cumulative_me,var_31_cumulative_me,var_32_cumulative_me,var_33_cumulative_me,var_34_cumulative_me,var_35_cumulative_me,var_36_cumulative_me,var_37_cumulative_me,var_38_cumulative_me,var_39_cumulative_me,var_40_cumulative_me,var_41_cumulative_me,var_42_cumulative_me,var_43_cumulative_me,var_44_cumulative_me,var_45_cumulative_me,var_46_cumulative_me,var_47_cumulative_me,var_48_cumulative_me,var_49_cumulative_me,var_50_cumulative_me,var_51_cumulative_me,var_52_cumulative_me,var_53_cumulative_me,var_54_cumulative_me,var_55_cumulative_me,var_56_cumulative_me,var_57_cumulative_me,var_58_cumulative_me,var_59_cumulative_me,var_60_cumulative_me,var_61_cumulative_me,var_62_cumulative_me,var_63_cumulative_me,var_64_cumulative_me,var_65_cumulative_me,var_66_cumulative_me,var_67_cumulative_me,var_68_cumulative_me,var_69_cumulative_me,var_70_cumulative_me,var_71_cumulative_me,var_72_cumulative_me,var_73_cumulative_me,var_74_cumulative_me,var_75_cumulative_me,var_76_cumulative_me,var_77_cumulative_me,var_78_cumulative_me,var_79_cumulative_me,var_80_cumulative_me,var_81_cumulative_me,var_82_cumulative_me,var_83_cumulative_me,var_84_cumulative_me,var_85_cumulative_me,var_86_cumulative_me,var_87_cumulative_me,var_88_cumulative_me,var_89_cumulative_me,var_90_cumulative_me,var_91_cumulative_me,var_92_cumulative_me,var_93_cumulative_me,var_94_cumulative_me,var_95_cumulative_me,var_96_cumulative_me,var_97_cumulative_me,var_98_cumulative_me,var_99_cumulative_me,var_100_cumulative_me,...,var_103_cumulative_me,var_104_cumulative_me,var_105_cumulative_me,var_106_cumulative_me,var_107_cumulative_me,var_108_cumulative_me,var_109_cumulative_me,var_110_cumulative_me,var_111_cumulative_me,var_112_cumulative_me,var_113_cumulative_me,var_114_cumulative_me,var_115_cumulative_me,var_116_cumulative_me,var_117_cumulative_me,var_118_cumulative_me,var_119_cumulative_me,var_120_cumulative_me,var_121_cumulative_me,var_122_cumulative_me,var_123_cumulative_me,var_124_cumulative_me,var_125_cumulative_me,var_126_cumulative_me,var_127_cumulative_me,var_128_cumulative_me,var_129_cumulative_me,var_130_cumulative_me,var_131_cumulative_me,var_132_cumulative_me,var_133_cumulative_me,var_134_cumulative_me,var_135_cumulative_me,var_136_cumulative_me,var_137_cumulative_me,var_138_cumulative_me,var_139_cumulative_me,var_140_cumulative_me,var_141_cumulative_me,var_142_cumulative_me,var_143_cumulative_me,var_144_cumulative_me,var_145_cumulative_me,var_146_cumulative_me,var_147_cumulative_me,var_148_cumulative_me,var_149_cumulative_me,var_150_cumulative_me,var_151_cumulative_me,var_152_cumulative_me,var_153_cumulative_me,var_154_cumulative_me,var_155_cumulative_me,var_156_cumulative_me,var_157_cumulative_me,var_158_cumulative_me,var_159_cumulative_me,var_160_cumulative_me,var_161_cumulative_me,var_162_cumulative_me,var_163_cumulative_me,var_164_cumulative_me,var_165_cumulative_me,var_166_cumulative_me,var_167_cumulative_me,var_168_cumulative_me,var_169_cumulative_me,var_170_cumulative_me,var_171_cumulative_me,var_172_cumulative_me,var_173_cumulative_me,var_174_cumulative_me,var_175_cumulative_me,var_176_cumulative_me,var_177_cumulative_me,var_178_cumulative_me,var_179_cumulative_me,var_180_cumulative_me,var_181_cumulative_me,var_182_cumulative_me,var_183_cumulative_me,var_184_cumulative_me,var_185_cumulative_me,var_186_cumulative_me,var_187_cumulative_me,var_188_cumul

In [8]:
tmp = pd.DataFrame()
tmp['target'] = train.target
tmp['target_encode_sum'] = cum_target_encode_train['target_encode_sum']
tmp.corr('spearman')

,target,target_encode_sum
target,1.000000,0.481714
target_encode_sum,0.481714,1.000000


In [10]:
cum_target_encode_train.to_pickle('features/cum_target_encode_train.pkl')
cum_target_encode_test.to_pickle('features/cum_target_encode_test.pkl')